## Generate Text

In [1]:
import torch
from watermark.auto_watermark import AutoWatermark
from utils.transformers_config import TransformersConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
# Device
device = "cuda" if torch.cuda.is_available() else "cpu"


/home/aroma/miniconda3/envs/wm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
transformers_config = TransformersConfig(
    model=AutoModelForCausalLM.from_pretrained('facebook/opt-1.3b').to(device),
    tokenizer=AutoTokenizer.from_pretrained('facebook/opt-1.3b'),
    vocab_size=50272,
    device=device,
    max_new_tokens=200,
    min_length=204,
    do_sample=True,
    no_repeat_ngram_size=4
)

In [3]:
 
# Load watermark algorithm
myWatermark = AutoWatermark.load(
    'KGW', 
    algorithm_config='config/KGW.json',
    transformers_config=transformers_config
)


In [4]:
# Prompt
prompt = 'Good Afternoon.'

# Generate and detect
watermarked_text = myWatermark.generate_watermarked_text(prompt)
wm_detect_result = myWatermark.detect_watermark(watermarked_text)
unwatermarked_text = myWatermark.generate_unwatermarked_text(prompt)
unwm_detect_result = myWatermark.detect_watermark(unwatermarked_text)


In [5]:
print(watermarked_text)
print(wm_detect_result)


Good Afternoon. Is there anything other than the weekly quests to farm mats besides buying the cheap and weak mats from vendors to upgrade my guns.  I'm a little new to the game trying to catch'em all but all I seem to be able to get is materials for some guns.  Can anyone help?
Honestly, just do the daily daily daily daily missions. That should help, and if you do those daily (more or less), you get enough to lvl up all the guns you get. (assuming you're talking about gun crafting mats), which will level you up more, and increase the item level.  If not, buy a few daily "quests" from NPCs to increase the item  level.
> If not, *buy the cheap and strong* **mats** from vendors toUpgrade my guns
Ah, I see. I think the mats you get from leveling up guns are not that cheap, if they're in fact that cheap, and have no effect on item
{'is_watermarked': True, 'score': 9.89625992944471}


In [6]:
print(unwatermarked_text)
print(unwm_detect_result)

Good Afternoon. Today, we'll discuss the different options out there for making the right decision when it comes to the stock market, along with some of the factors that need to be considered before investing. Let us begin with a quick overview of the recent market events, including what exactly caused them...

About Us

The team behind the site:

David and Jim have been providing realtor/home owner information (both consumer based and for real estate professionals) since March, 2006. This was done as a way to help new home owners and real estate professionals make sound investment and buying decisions. Their mission now is to share their research with users of this community site for all their real estate needs.

All information posted herein is deemed reliable but is not guaranteed to be accurate.
User should independently confirm all information before relying on it. This site should not be used for offering personal financial advice. Always consult a professional for investment, ta

In [7]:
with open('KGW_test.output.txt','w') as f:
    l=[watermarked_text,wm_detect_result,unwatermarked_text,unwm_detect_result]
    l=[str(item) for item in l]
    f.write('\n'.join(l))

## Visualization

In [8]:
import torch
from visualize.font_settings import FontSettings
from watermark.auto_watermark import AutoWatermark
from utils.transformers_config import TransformersConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from visualize.visualizer import DiscreteVisualizer
from visualize.legend_settings import DiscreteLegendSettings
from visualize.page_layout_settings import PageLayoutSettings
from visualize.color_scheme import ColorSchemeForDiscreteVisualization


In [9]:

# Get data for visualization
watermarked_data = myWatermark.get_data_for_visualization(watermarked_text)
unwatermarked_data = myWatermark.get_data_for_visualization(unwatermarked_text)

# Init visualizer
visualizer = DiscreteVisualizer(color_scheme=ColorSchemeForDiscreteVisualization(),
                                font_settings=FontSettings(), 
                                page_layout_settings=PageLayoutSettings(),
                                legend_settings=DiscreteLegendSettings())
# Visualize
watermarked_img = visualizer.visualize(data=watermarked_data, 
                                       show_text=True, 
                                       visualize_weight=True, 
                                       display_legend=True)

unwatermarked_img = visualizer.visualize(data=unwatermarked_data,
                                         show_text=True, 
                                         visualize_weight=True, 
                                         display_legend=True)


AttributeError: 'ImageDraw' object has no attribute 'textsize'

In [ ]:

# Save
watermarked_img.save("KGW_watermarked.png")
unwatermarked_img.save("KGW_unwatermarked.png")